In [31]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Input,Conv2D,MaxPooling2D,Flatten,Dense,BatchNormalization,Dropout
from keras.callbacks import EarlyStopping

In [12]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = 'cat_dog_train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

test_ds = keras.utils.image_dataset_from_directory(
    directory = 'cat_dog_test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [15]:
# Normalization
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

In [27]:
model = Sequential()

model.add(Input((256,256,3)))

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [28]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 batch_normalization_10 (Ba  (None, 254, 254, 32)      128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_11 (Ba  (None, 125, 125, 64)      256       
 tchNormalization)                                               
                                                      

In [29]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [33]:
callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.1,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True
)

In [34]:
history = model.fit(train_ds,epochs=20,validation_data=test_ds, callbacks=callback)

Epoch 1/20
625/625 [==============================] - 1004s 2s/step - loss: 2.2907 - accuracy: 0.5245 - val_loss: 0.6701 - val_accuracy: 0.5436
Epoch 2/20
593/625 [===========================>..] - ETA: 18s - loss: 0.7015 - accuracy: 0.5324

KeyboardInterrupt: 